In [44]:
import pandas as pd
import json
import re

# Load your dataset
df = pd.read_csv("train.csv")

print(df.head())

                                       conversations          id
0  [{'from': 'human', 'value': "I've been feeling...  identity_0
1  [{'from': 'human', 'value': "Hi, I'm feeling r...  identity_1
2  [{'from': 'human', 'value': "Hey, I hope you'r...  identity_2
3  [{'from': 'human', 'value': "I'm feeling reall...  identity_3
4  [{'from': 'human', 'value': "I'm feeling reall...  identity_4


In [45]:
# print(df.head().to_dict())

{'conversations': {0: '[{\'from\': \'human\', \'value\': "I\'ve been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me."}\n {\'from\': \'gpt\', \'value\': "Hey there, I\'m here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what\'s been going on?"}\n {\'from\': \'human\', \'value\': "I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It\'s been a really moving experience for me."}\n {\'from\': \'gpt\', \'value\': "I can understand how it can be overwhelming when we\'re faced with higher expectations. It\'s okay to acknowledge your emotions and allow yourself to feel sad in this situation. It\'s an important part of the healing process. What specific challenges have you been facing at work?"}\n {\'from\': \'human\', \'value\': "Well, the workload has increased significantly, 

In [46]:
import ast

In [47]:
def parse_conversation_flexible(conv_str):
    try:
        conv_str_fixed = re.sub(r"}\s*{", "}, {", conv_str)  # Fix missing commas between dicts
        return ast.literal_eval(conv_str_fixed)  # Parse to list
    except:
        return []

df["parsed_conversations"] = df["conversations"].apply(parse_conversation_flexible)

In [48]:
# Step 2: Extract patient-therapist pairs
def extract_grouped_dialogue(conversation, identity):
    dialogue_pairs = []
    for i in range(len(conversation) - 1):
        if conversation[i]['from'] == 'human' and conversation[i+1]['from'] == 'gpt':
            pair = {
                "patient": conversation[i]['value'],
                "therapist": conversation[i+1]['value']
            }
            dialogue_pairs.append(pair)
    return {
        "identity": identity,
        "dialogue": dialogue_pairs
    }

In [49]:
grouped_conversations = []
for idx, row in df.iterrows():
    parsed = row["parsed_conversations"]
    identity = row["id"]
    grouped_convo = extract_grouped_dialogue(parsed, identity)
    grouped_conversations.append(grouped_convo)

In [28]:
# Extract grouped dialogue format
def extract_grouped_dialogue(conversation, identity):
    dialogue_pairs = []
    for i in range(len(conversation) - 1):
        if conversation[i]['from'] == 'human' and conversation[i+1]['from'] == 'gpt':
            pair = {
                "patient": conversation[i]['value'],
                "therapist": conversation[i+1]['value']
            }
            dialogue_pairs.append(pair)
    return {
        "identity": identity,
        "dialogue": dialogue_pairs
    }

In [29]:
# Apply to full dataframe
grouped_conversations = []
for idx, row in df.iterrows():
    grouped_convo = extract_grouped_dialogue(row["parsed_conversations"], row["id"])
    grouped_conversations.append(grouped_convo)

In [30]:
grouped_conversations[:1]

[{'identity': 'identity_0', 'dialogue': []}]

In [50]:
# Step 4: Preview first result
print(json.dumps(grouped_conversations[0], indent=2))

{
  "identity": "identity_0",
  "dialogue": [
    {
      "patient": "I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me.",
      "therapist": "Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on?"
    },
    {
      "patient": "I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It's been a really moving experience for me.",
      "therapist": "I can understand how it can be overwhelming when we're faced with higher expectations. It's okay to acknowledge your emotions and allow yourself to feel sad in this situation. It's an important part of the healing process. What specific challenges have you been facing at work?"
    },
    {
      "patient": "Well, the workload has increased significantly, and I find it hard to maintain a wo

In [51]:
import json

# File path to save your output
output_path = "grouped_conversations.jsonl"

# Write each conversation as a JSON line
with open(output_path, "w", encoding="utf-8") as f:
    for convo in grouped_conversations:
        json.dump(convo, f)
        f.write("\n")

print(f"Saved {len(grouped_conversations)} conversations to {output_path}")


Saved 99086 conversations to grouped_conversations.jsonl


In [53]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 15.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openai
import json
from openai import OpenAI

# Set your OpenAI API key here
client = OpenAI(api_key="UPDATE YOUR OWN KEY BABE")

# File paths
input_file = "grouped_conversations.jsonl"
output_file = "grouped_conversations_with_cot_final.jsonl"

In [9]:
# prompt template
def build_prompt(patient, therapist):
#     return f"""You are a compassionate and emotionally intelligent therapist.

# Below is a message from a user and the response given by the therapist.

# Your task is to explain the internal thought process the therapist might have gone through before giving the response. The explanation should justify the actual response, not invent a new one.

# Be concise (1 sentence), emotionally aware, and consistent with the tone. The thought process must detail on the issues or the query that the user has given, and ‘think’ in a way that leads to the response that is already present.

# User: {patient}

# Therapist's Response: {therapist}

# Therapist’s Thought Process:"""


    return f""" 
user: {patient}
therapist: {therapist}

You are a compassionate and emotionally intelligent therapist assistant. Your task is to analyze the provided user's message(above), think step-by-step about what the therapist should consider before replying after understanding the user's query. The reasoning must comply with the therapist’s response (above). Just give 1-2 sentences for the reasoning, and it must be exactly what the therapist thinks before responding in 1st person voice."""

In [12]:
output_data = []

# Open input file and process
with open(input_file, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        if idx >= 1500:  # just first few conversation for now
            break
        if idx % 50 == 0:
            print(f"Inflated {idx} records: {idx / 15} %")
        convo = json.loads(line)
        identity = convo["identity"]
        dialogue = convo["dialogue"]

        new_dialogue = []
        for turn in dialogue:
            patient = turn["patient"]
            therapist = turn["therapist"]
            prompt = build_prompt(patient, therapist)

            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.5,
                    max_tokens=100
                )
                # print(f"response: {response}")
                reasoning = response.choices[0].message.content.strip()
            except Exception as e:
                print(f"❌ Error for index: {idx},  message: {e}")
                reasoning = "N/A"

            new_dialogue.append({
                "patient": patient,
                "reasoning": reasoning,
                "therapist": therapist
            })

        output_data.append({
            "identity": identity,
            "dialogue": new_dialogue
        })


Inflated 0 records: 0.0 %
Inflated 50 records: 3.3333333333333335 %
Inflated 100 records: 6.666666666666667 %
Inflated 150 records: 10.0 %
Inflated 200 records: 13.333333333333334 %
Inflated 250 records: 16.666666666666668 %
Inflated 300 records: 20.0 %
Inflated 350 records: 23.333333333333332 %
Inflated 400 records: 26.666666666666668 %
Inflated 450 records: 30.0 %
Inflated 500 records: 33.333333333333336 %
Inflated 550 records: 36.666666666666664 %
Inflated 600 records: 40.0 %
Inflated 650 records: 43.333333333333336 %
Inflated 700 records: 46.666666666666664 %
Inflated 750 records: 50.0 %
Inflated 800 records: 53.333333333333336 %
Inflated 850 records: 56.666666666666664 %
Inflated 900 records: 60.0 %
Inflated 950 records: 63.333333333333336 %
Inflated 1000 records: 66.66666666666667 %
Inflated 1050 records: 70.0 %
Inflated 1100 records: 73.33333333333333 %
Inflated 1150 records: 76.66666666666667 %
Inflated 1200 records: 80.0 %
Inflated 1250 records: 83.33333333333333 %
Inflated 13

In [13]:
# Save output
with open(output_file, "w", encoding="utf-8") as f_out:
    for entry in output_data:
        json.dump(entry, f_out)
        f_out.write("\n")

print(f"✅ Saved {len(output_data)} conversations with CoT to {output_file}")

✅ Saved 1500 conversations with CoT to grouped_conversations_with_cot_final.jsonl


In [1]:
print("""
Therapist Chatbot (type 'quit' to exit)

Patient: Hi, I'm feeling very down today. I feel a lot of stress for exam and the deadlines.

(Thinking...)
Therapist Thought Process: I need to create a safe space for Charlie to express his feelings and concerns about his workload and deadlines. It's important to encourage him to elaborate on the specific stressors so I can better understand his situation and provide appropriate support.

Therapist: Hi Charlie, I'm sorry to hear that you're feeling down. It sounds like you're going through a tough time. Can you tell me more about what specifically is causing you stress?

Patient: Yes, there is a lot of deadlines nearby and my work is not done as I procrastinated a lot.

(Thinking...)
Therapist Thought Process: 

Therapist: I need to validate Charlie's feelings of stress and ensure he feels heard and understood. I should also encourage him to share specific situations or specific challenges he's facing that contribute to his workload.
""")


Therapist Chatbot (type 'quit' to exit)

Patient: Hi, I'm feeling very down today. I feel a lot of stress for exam and the deadlines.

(Thinking...)
Therapist Thought Process: I need to create a safe space for Charlie to express his feelings and concerns about his workload and deadlines. It's important to encourage him to elaborate on the specific stressors so I can better understand his situation and provide appropriate support.

Therapist: Hi Charlie, I'm sorry to hear that you're feeling down. It sounds like you're going through a tough time. Can you tell me more about what specifically is causing you stress?

Patient: Yes, there is a lot of deadlines nearby and my work is not done as I procrastinated a lot.

(Thinking...)
Therapist Thought Process: 

Therapist: I need to validate Charlie's feelings of stress and ensure he feels heard and understood. I should also encourage him to share specific situations or specific challenges he's facing that contribute to his workload.

